In [26]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

from src.QAOA import QAOA
from src.Pennylane_QAOA import Pennylane_QAOA
from src.Cirq_QAOA import Cirq_QAOA
from src.Qsimcirq_QAOA import Qsimcirq_QAOA
from src.Spynoza_QAOA import Spynoza_QAOA

from src.Chain import Chain
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       qubo_limits, 
                       check_qubo)

In [27]:
n_qubits = 10
init_strat = np.array([1 if i%2 == 1 else 0 for i in range(n_qubits)])
k = n_qubits // 2
my_indices = [(i, i+1) for i in range(n_qubits-1)]
n_layers = 4
alpha = 0.5
seed = 0

max_iter = 100

In [28]:
 # Defining topology
my_chain = Chain(N_qubits=n_qubits)
my_chain.set_initialization_strategy(strategy=init_strat)

# Deciding between grid and 1d chain topology
my_topology = my_chain

# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=n_qubits, seed=seed)

# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=n_qubits,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
#full_space_max_cost = full_result['c_max']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])

# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])


if not portfolio_subspace_min_state_str == qubo_min_state_str:
    raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
    raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not qubo_max_cost >= portfolio_subspace_max_cost:
    raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')

In [29]:
qiskit_ansatz = QAOA(N_qubits=n_qubits,
                     layers=n_layers,
                     QUBO_matrix=Q,
                     QUBO_offset=offset,
                     constraining_mixer=False,
                     Topology=my_topology)

pennylane_ansatz = Pennylane_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

qsim_ansatz = Qsimcirq_QAOA(N_qubits=n_qubits,
                            layers=n_layers,
                            QUBO_matrix=Q,
                            QUBO_offset=offset)


cirq_ansatz = Cirq_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

spynoza_ansatz = Spynoza_QAOA(N_qubits=n_qubits,
                                  layers=n_layers,
                                  QUBO_matrix=Q,
                                  QUBO_offset=offset)

In [30]:
# Choosing optimizer for scipy
available_methods = ['COBYLA', 'Nelder-Mead', 'BFGS']
optimizer_method = available_methods[0]

# Generating initial guess for rotation angles
np.random.seed(seed)
theta_min, theta_max = -2*np.pi, 2*np.pi
N_angles = 2 * n_layers
theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_angles)

In [31]:
qiskit_res = sc.optimize.minimize(fun=qiskit_ansatz.get_cost, 
                                  x0=theta_i,
                                  method=optimizer_method,
                                  options={'disp': False, 
                                           'maxiter': max_iter})
qiskit_res


 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.3679089695788974
       x: [ 1.613e+00  2.706e+00  1.505e+00  1.681e+00 -9.631e-01
            1.837e+00 -7.859e-01  6.057e+00]
    nfev: 100
   maxcv: 0.0

In [7]:
pennylane_res = sc.optimize.minimize(fun=pennylane_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter})
pennylane_res

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.17690351219608175
       x: [ 1.919e+00  3.667e+00  1.344e+00  5.440e-01 -9.589e-01
            1.842e+00 -5.014e-01  4.741e+00]
    nfev: 100
   maxcv: 0.0

In [8]:
qsim_res = sc.optimize.minimize(fun=qsim_ansatz.get_cost, 
                                  x0=theta_i,
                                  method=optimizer_method,
                                  options={'disp': False, 
                                           'maxiter': max_iter})
qsim_res

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.19887508720350228
       x: [ 4.240e-01  2.957e+00  1.292e+00  6.869e-01 -9.979e-01
            2.778e+00 -8.528e-01  4.907e+00]
    nfev: 100
   maxcv: 0.0

In [9]:
cirq_res = sc.optimize.minimize(fun=cirq_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter})
cirq_res

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.19310186768955434
       x: [ 4.446e-01  2.715e+00  1.509e+00  3.816e-01 -9.547e-01
            2.798e+00 -6.918e-01  4.948e+00]
    nfev: 100
   maxcv: 0.0

In [32]:
spynoza_res = sc.optimize.minimize(fun=spynoza_ansatz.get_cost, 
                                     x0=theta_i,
                                     method=optimizer_method,
                                     options={'disp': False, 
                                               'maxiter': max_iter})
spynoza_res

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.3827159327129277
       x: [ 5.826e-01  2.131e+00  1.070e+00  3.366e-01 -1.429e+00
            1.711e+00 -1.283e+00  4.635e+00]
    nfev: 100
   maxcv: 0.0

In [25]:
31.9/94

0.33936170212765954